In [2]:
%pylab inline
import pandas as pd
import numpy as np
from IPython.display import display
import re

Populating the interactive namespace from numpy and matplotlib


## Data

In [1]:
file_name = '/dataset/personal_projects/lsm/20160123_1차 발주_3280.xlsx'

## Settings

In [6]:
N_DAS = 96
SHEET_NAME = 'original'
FINDING_LOCATION_TIME = 9
FINDING_PRODUCT_TIME = 12

## Program

In [4]:
def display_time(seconds):
    response = ''
    hours = floor(seconds/int(60*60))
    seconds -= hours*60*60
    if hours >= 1:
        response += '%d시간 ' % hours
    
    minutes = floor(seconds/int(60))
    seconds -= minutes*60
    if minutes >= 1:
        response += '%d분 ' % minutes
    
    response += '%d초' % seconds
    return response


def display_result(n_epoch, n_location, n_product, n_barcode, is_possible=None):
    cost_location = n_location * FINDING_LOCATION_TIME
    cost_product = n_barcode * FINDING_PRODUCT_TIME
    if is_possible is not None:
        print('시뮬레이션 결과:\t', is_possible)
    
    print('차수 횟수:\t %d개' % n_epoch)
    print('길찾기 횟수:\t %d건' % n_location)
    print('상품찾기 횟수:\t %d건' % n_product)
    print('바코드찾기 횟수:\t %d건' % n_barcode)
    print('총 찾기 횟수:\t %d건' % int(n_location + n_barcode))
    print('갈찾기 소요시간:\t', display_time(cost_location))
    print('바코드찾기소요시간:\t', display_time(cost_product) )
    print('총 소요시간:\t', display_time(cost_location + cost_product))    

In [77]:
class DAS(object):
    def __init__(self, n_das=N_DAS):
        self.N_DAS = n_das
    
    def initialize(self, data, codes):
        self.data = data
        self.codes = codes
        
    def process(self, item):
        pass
    

class HiLSM(object):
    EMPTY_LOCATION = 'Z1-1'
    
    def __init__(self, n_das=N_DAS, sheet=SHEET_NAME):
        self.data = None
        self.N_DAS = n_das
        self.sheet = SHEET_NAME
        self.das = DAS(n_das=n_das)
        
    def load(self, file_name):
        COL_NAMES = ['date', 'order_number', 'brand_code', 'brand_name', 
                     'product', 'property', 'location', 'barcode', 'n', 'price', 
                     'status', 'orderer', 'recipient', 'address', 'postcode', 
                     'contact1', 'contact2', 'comment', 'part_delivery', 'code']
        self.data = pd.read_excel(file_name, 
                                  names=COL_NAMES,
                                  sheetname=self.sheet)
        
    def preprocess(self):
        # Pre-Process missing locations
        self.data['location'] = self.data['location'].fillna(self.EMPTY_LOCATION)
        
        # Split Locations to l1, l2, l3
        location_regex = '(?P<l1>[a-zA-Z]+)-?(?P<l2>\d+)-(?P<l3>\d+)'
        self.data['location'][~self.data['location'].str.contains(location_regex)] = self.EMPTY_LOCATION
        location_series = self.data['location'].str.extract(location_regex)
        self.data = pd.concat([self.data, location_series], axis=1)
    
        
    def process_by_code(self):
        most_n = self.data[['code', 'n']].groupby(by='code').sum()
        most_n = most_n.sort_values(by='n', ascending=False)
        codes = most_n.index.unique().values
        
        return self.split_codes(codes)
    
    def process_by_product(self):
        most_products = self.data[['product', 'n']].groupby(by='product').sum()
        most_products = most_products.sort_values(by='n', ascending=False)
        codes = []
        for product in most_products.index:
            for code in self.data[self.data['product'] == product]['code'].values:
                if code not in codes:
                    codes.append(code)
        
        codes = np.array(codes)
        return self.split_codes(codes)
        
    def split_codes(self, codes):
        response = []
        for i in range(0, self.data.shape[0], self.N_DAS):
            r = codes[i: i+self.N_DAS]
            if len(r) >= 1:
                response.append(codes[i: i+self.N_DAS])
        return response
        
        
    def evaluate(self, bunch_codes):
        n_epoch = 0
        n_locations = 0
        n_products = 0
        n_barcodes = 0
        
        for codes in bunch_codes:
            data = self.data[self.data['code'].isin(codes)]
        
            n_location, n_product, n_barcode = self._calculate_n_times(data)
            n_locations += n_location
            n_products += n_product
            n_barcodes += n_barcode
            n_epoch += 1
            
        return n_epoch, n_locations, n_products, n_barcodes
            
            
    def _calculate_n_times(self, data):
        data = data.sort_values(['l1', 'l2', 'l3', 'barcode'])
        n_location = 0
        n_product = 0
        n_barcode = 0
        
        current_location = None
        current_product = None
        current_barcode = None
        
        for idx, item in data.iterrows():
            l = item['l1'] + str(item['l2'])
            product = item['product']
            barcode = item['barcode']
            
            if current_location != l:
                current_location = l
                n_location += 1
            
            if current_product != product:
                current_product = product
                n_product += 1
                
            if current_barcode != barcode:
                current_barcode = barcode
                n_barcode += 1
        
        return n_location, n_product, n_barcode
    
    def is_possible(self, bunch_codes):
        # 원래 데이터의 row count가 동일한지 체크
        if self.data.isin(np.concatenate(bunch_codes)).shape != self.data.shape:
            return False
        
        for codes in bunch_codes:
            if codes.shape[0] > self.N_DAS:
                return False
            
            # DAS Initilization
            self.das.initialize(self.data, codes)
            
            # Load 차수
            data = self.data[self.data['code'].isin(codes)]
            for idx, item in data.iterrows():
                self.das.process(item)
            
        return True
    
    def process_by_location(self):
        print(len(self.data['product'].unique()))
        print(len(self.data['product']))
    
    
        
lsm = HiLSM()
lsm.load(file_name)
lsm.preprocess()
lsm.process_by_location()

727
3279


/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:37: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:38: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


### 주문 단위로 찾을때의 비용

주문이란 "관리번호"를 기준으로 하여 한건 한건씩 찾을때 걸리는 시간입니다.<br>
"제품 로케이션"을 기준으로 optimize되어 있기 때문에 실제보다 조금더 빠를수 있습니다.

In [78]:
codes = lsm.process_by_code()
analytics = lsm.evaluate(codes)
is_possible = lsm.is_possible(codes)
display_result(*analytics, is_possible=is_possible)

시뮬레이션 결과:	 True
차수 횟수:	 18개
길찾기 횟수:	 1017건
상품찾기 횟수:	 2146건
바코드찾기 횟수:	 2621건
총 찾기 횟수:	 3638건
갈찾기 소요시간:	 2시간 32분 33초
바코드찾기소요시간:	 8시간 44분 12초
총 소요시간:	 11시간 16분 45초


### 상품 단위로 찾을때의 비용

가장많이 나온 상품을 기준으로 최대한 한꺼번에 동일한 상품들을 가져오도록 optimize하는 방법입니다.

In [79]:
codes = lsm.process_by_product()
analytics = lsm.evaluate(codes)
is_possible = lsm.is_possible(codes)
display_result(*analytics, is_possible=is_possible)

시뮬레이션 결과:	 True
차수 횟수:	 18개
길찾기 횟수:	 901건
상품찾기 횟수:	 1552건
바코드찾기 횟수:	 1968건
총 찾기 횟수:	 2869건
갈찾기 소요시간:	 2시간 15분 9초
바코드찾기소요시간:	 6시간 33분 36초
총 소요시간:	 8시간 48분 45초
